In [249]:
%%capture
!pip install mne
!pip install pytorch-lightning

In [250]:
%%capture
#!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/fshy54ypyh-1.zip -O data.zip
#!unzip data.zip


In [251]:
from glob import glob
import scipy.io
import torch.nn as nn
import torch
import numpy as np
import mne

In [252]:
input=torch.randn(3,19,512)
input.shape

torch.Size([3, 19, 512])

In [253]:
class Block(nn.Module):
    def __init__(self,inplace):
        super().__init__()
        self.conv1=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=2,stride=2,padding=0)
        self.conv2=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=4,stride=2,padding=1)
        self.conv3=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=8,stride=2,padding=3)
        self.relu=nn.ReLU()

    def forward(self,x):
        x1=self.relu(self.conv1(x))
        x2=self.relu(self.conv2(x))
        x3=self.relu(self.conv3(x))
        x=torch.cat([x1,x3,x3],dim=1)
        return x

In [254]:
class ChronoNet(nn.Module):
    def __init__(self,channel):
        super().__init__()
        self.block1=Block(channel)
        self.block2=Block(96)
        self.block3=Block(96)
        self.gru1=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
        self.gru2=nn.GRU(input_size=32,hidden_size=32,batch_first=True)
        self.gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
        self.gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
        self.gru_linear=nn.Linear(64,1)
        self.flatten=nn.Flatten()
        self.fc1=nn.Linear(32,1)
        self.relu=nn.ReLU()
    def forward(self,x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.block3(x)
        x=x.permute(0,2,1)
        gru_out1,_=self.gru1(x)
        gru_out2,_=self.gru2(gru_out1)
        gru_out=torch.cat([gru_out1,gru_out2],dim=2)
        gru_out3,_=self.gru3(gru_out)
        gru_out=torch.cat([gru_out1,gru_out2,gru_out3],dim=2)
        #print('gru_out',gru_out.shape)
        linear_out=self.relu(self.gru_linear(gru_out.permute(0,2,1)))
        gru_out4,_=self.gru4(linear_out.permute(0,2,1))
        x=self.flatten(gru_out4)
        x=self.fc1(x)
        return x

In [255]:
input=torch.randn(3,19,512)
input.shape
model=ChronoNet(19)
out=model(input)
out.shape

torch.Size([3, 1])

In [256]:
# IDD_data_path='/content/Data/CleanData/CleanData_TDC/Rest'
# TDC_data_path='/content/Data/Data/CleanData/Data/Data/CleanData/CleanData_IDD/Rest'
# !rm '/content/Data/Data/CleanData/Data/Data/CleanData/CleanData_IDD/Rest/NDS001_Rest_CD(1).mat'
path=glob('dataverse_files-2/*.edf')
len(path)
healthyPath=[i for i in path if 'h' in i.split("/")[1]]
PatientPath=[i for i in path if 's' in i.split("/")[1]]

In [257]:
# def convertmat2mne(data):
#   ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
#   ch_types = ['eeg'] * 14
#   sampling_freq=128
#   info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
#   info.set_montage('standard_1020')
#   data=mne.io.RawArray(data, info)
#   data.set_eeg_reference()
#   data.filter(l_freq=1,h_freq=30)
#   epochs=mne.make_fixed_length_epochs(data,duration=4,overlap=0)
#   return epochs.get_data()
def read_data(filepath):
    data=mne.io.read_raw_edf(filepath,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=1,h_freq=30)
    epoch=mne.make_fixed_length_epochs(data,duration=4,overlap=1)
    arr=epoch.get_data()
    return arr

In [258]:
# %%capture
# idd_subject=[]
# for idd in glob(IDD_data_path+'/*.mat'):
#   data=scipy.io.loadmat(idd)['clean_data']
#   data=convertmat2mne(data)
#   idd_subject.append(data)
healthy_epochArr=[read_data(i) for i in healthyPath]
patient_epochArr=[read_data(i) for i in PatientPath]
control_epoch_labels=[len(i)*[0] for i in healthy_epochArr]
patient_epoch_labels=[len(i)*[1] for i in patient_epochArr]

Extracting EDF parameters from /Users/antotomabraham/Documents/Developement/SET/dataverse_files-2/h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
308 matching events found
No baseline correction applied
0 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1000 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /Users/antotomabraham/Documents/Developement/SET/dataverse_files-2/h02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband e

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
303 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 303 events and 1000 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /Users/antotomabraham/Documents/Developement/SET/dataverse_files-2/h13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 241249  =      0.000 ...   

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Not setting metadata
303 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 303 events and 1000 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /Users/antotomabraham/Documents/Developement/SET/dataverse_files-2/s09.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 296249  =      0.000 ...  1184.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband e

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
394 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 394 events and 1000 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


In [259]:
# %%capture
# tdc_subject=[]
# for tdc in glob(TDC_data_path+'/*.mat'):
#   data=scipy.io.loadmat(tdc)['clean_data']
#   data=convertmat2mne(data)
#   tdc_subject.append(data)
data_list=healthy_epochArr+patient_epochArr
label_list=control_epoch_labels+patient_epoch_labels
group_list=[[i]*len(j) for i,j in enumerate (data_list)]

In [260]:
#len(idd_subject),len(tdc_subject)

In [261]:
# control_epochs_labels=[len(i)*[0] for i in tdc_subject]
# patients_epochs_labels=[len(i)*[1] for i in idd_subject]
# print(len(control_epochs_labels),len(patients_epochs_labels))

In [262]:
# data_list=tdc_subject+idd_subject
# label_list=control_epochs_labels+patients_epochs_labels
# groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
# print(len(data_list),len(label_list),len(groups_list))


In [263]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [264]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(group_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(9605, 1000, 19) (9605,) (9605,)


In [265]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
    train_features=np.moveaxis(train_features,1,2)
    val_features=np.moveaxis(val_features,1,2)

    break

In [266]:
train_features = torch.Tensor(train_features)
val_features = torch.Tensor(val_features)
train_labels = torch.Tensor(train_labels)
val_labels = torch.Tensor(val_labels)

In [267]:
len(val_features),len(val_labels)

(1943, 1943)

In [268]:
train_features.shape

torch.Size([7662, 19, 1000])

In [269]:
from pytorch_lightning import LightningModule,Trainer
import torchmetrics
from torch.utils.data import TensorDataset,DataLoader

In [270]:
class ChronoModel(LightningModule):
    def __init__(self):
        super(ChronoModel,self).__init__()
        self.model=ChronoNet(19)
        self.lr=1e-3
        self.bs=12
        self.worker=2
        self.acc=torchmetrics.Accuracy(task='BINARY')
        self.creterion=nn.BCEWithLogitsLoss()

    def forward(self,x):
        x=self.model(x)
        return x

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=self.lr)

    def train_dataloader(self):
        dataset=TensorDataset(train_features,train_labels)
        dataloader=DataLoader(dataset,batch_size=self.bs,num_workers=self.worker,shuffle=True)
        return dataloader

    def training_step(self,batch,batch_idx):
        signal,label=batch
        out=self(signal.float())
        loss=self.creterion(out.flatten(),label.float().flatten())
        acc=self.acc(out.flatten(),label.long().flatten())
        return {'loss':loss,'acc':acc}

    def trained_epoch_end(self,outputs):
        acc=torch.stack([x['acc'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        loss=torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        print('train acc loss',acc,loss)

    def val_dataloader(self):
        dataset=TensorDataset(val_features,val_labels)
        dataloader=DataLoader(dataset,batch_size=self.bs,num_workers=self.worker,shuffle=True)
        return dataloader

    def validation_step(self,batch,batch_idx):
        signal,label=batch
        out=self(signal.float())
        loss=self.creterion(out.flatten(),label.float().flatten())
        acc=self.acc(out.flatten(),label.long().flatten())
        return {'loss':loss,'acc':acc}

    def on_validation_epoch_end(self,outputs):
        acc=torch.stack([x['acc'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        loss=torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().numpy().round(2)
        print('val acc loss',acc,loss)
    











In [271]:
model=ChronoModel()

In [272]:
trainer=Trainer(max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [273]:
trainer.fit(model)


  | Name      | Type              | Params
------------------------------------------------
0 | model     | ChronoNet         | 135 K 
1 | acc       | BinaryAccuracy    | 0     
2 | creterion | BCEWithLogitsLoss | 0     
------------------------------------------------
135 K     Trainable params
0         Non-trainable params
135 K     Total params
0.542     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/Users/antotomabraham/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/antotomabraham/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:   0%|                       | 0/2 [00:00<?, ?it/s]

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [12, 125] but got: [12, 64].